In [1]:
from sklearn import tree
from sklearn.externals.six import StringIO
# import graphviz
import numpy as np
import csv 
import pydot
import graphviz
import math 
import collections 
from sklearn.tree import export_graphviz

keys = {'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5,  
      'F#': 6, 'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11} 
genreStrings = [] 
allFeaturesMatrix = [] 
allFeaturesDict = {} 
masterGenreDict = {} 
genreArrayDict = {} 
genreDict = {} 

#songToGenre = {} 

def readMusicData(filename) :
    # open a file
    with open(filename) as f:
        line = csv.reader(f)
        # loop over each row in the file
        rowNum = 0
        for row in line:
            if rowNum == 0 : 
                rowNum += 1
                continue
            # cast each value to a float
            features = []
            currFeature = 0
            nameID = ""
            for value in row :
                if currFeature ==0: 
                    if value not in genreStrings: 
                        genreStrings.append(value) 
#                     features.append(value)

                if (currFeature >= 4 and currFeature <= 8) or currFeature == 10 \
                    or currFeature == 11 or currFeature == 13 or currFeature == 14 \
                        or currFeature == 16: 
                        value = float(value)
                if currFeature == 15 : 
                    value = int(value[0])
                if currFeature == 3 : 
                    nameID = str(value)
                if currFeature == 9 : 
                    value = keys[value]
                if currFeature == 12 : 
                    value = 1 if value == "Major" else 0
                features.append(value)
                currFeature += 1
            allFeaturesDict[nameID] = features
            allFeaturesMatrix.append(features)
            rowNum += 1
        # store the row into our dict

def requests(name) :
    songRequests = collections.defaultdict(None)
    count = 0
    with open(name,'r') as file :
        for line in file: 
            if line == "\n":
                break
            line = line.split(' = ')
            name = line[0]
            line[1] = line[1].replace('\"', "")
            ids = line[1].split(", ")
            ids[0] = ids[0].replace('c(', "")
            ids[4] = ids[4].replace(')\n', "")
            songRequests[name] = ids
            count += 1
            if count == 51 : break
    return collections.OrderedDict(sorted(songRequests.items()))
def makeGenreDict(filename,genreArray) : 
    with open(filename) as f: 
        line = csv.reader(f)
        rowNum = 0 
        for row in line: 
            if rowNum == 0 :  
                rowNum += 1 
                continue 
              # cast each value to a float 
              # features = [] 
            currFeature = 0 
            nameID = ""   
            gname = ""  
            for value in row : 
                if currFeature ==0: 
                    gname = value 
                if currFeature == 3 :  
                    nameID = str(value) 
                currFeature+=1 
            genreDict[nameID] = str(gname) 
            genreArray.append(gname) 
            rowNum += 1 
  #extract numerical features only, with the first value as 'id' 

def numericalFeaturesOnly(allFeatures, toUse = []) :  
    newFeatures = [] 
    if not toUse : 
        for feature in allFeatures : 
            newFeatures.append(feature[4:]) 
    else :  
        for feature in allFeatures :  
            feature = [feature[i] for i in range(len(feature)) if i in toUse] 
            newFeatures.append(feature) 
    return newFeatures 

def outputResults(clf, toUse = [], average = True) : 
    necessaryDict = {} 
    totalSongs = 0 
    numIncorrect = 0  
  #     print(friends) 
    for person, songIds in friends.items() :  
  #         print(person) 
        songs = [] 
        for song in songIds :  
              #Using all songs 
            info = allFeaturesDict[song] 
            if not toUse :  
                songs.append(info[4:]) 
                necessaryDict[song] = info[4:] 
            else :  
                songs.append([info[i] for i in range(len(info)) if i in toUse]) 
                necessaryDict[song] = [info[i] for i in range(len(info)) if i in toUse] 
#             print(necessaryDict[song]) 
            totalSongs+=1 



        print("Songs for my friend "  + person) 
        print(" Input songs: " ) 
  #         print(songs) 
        for song in songIds :  
            print("\t\t" + allFeaturesDict[song][2] + " by " + allFeaturesDict[song][1])
            scores = [] 
            scoresDict = {} 
  #         for song in songIds: 
#             j=0 
            g = clf.predict(necessaryDict[song])
            if(g!=genreDict[song]):  
                if(genreDict[song]=='NULL'): 
                    numIncorrect-=2 
                    totalSongs-=1 
                numIncorrect+=1 
    print("\n" ) 
    return (numIncorrect, totalSongs) 

print("Reading File Data!\n"  ) 
# Our friend song interests.  
friends = requests('SongInput.txt') 
#populate matrix and dict with all features 
readMusicData('SpotifyFeatures.csv') 


desiredFeatures = [4,5,8,10,11,13,14,16] 
desFeat2 = [4, 5, 7, 8, 10, 11, 13, 14, 16] 
# desFeat3 = [4, 5, 7, 8, 13] 
#just the numerical features sans artist/titles/genre/id 
numericalFeatures = numericalFeaturesOnly(allFeaturesMatrix, desFeat2) 
# model = LogisticRegression() 
# model.fit(allFeaturesMatrix, genreArray) 
# print(allFeaturesMatrix) 
genreToModel = {} 

numIncorrectClass = 0 
totalClassified = 0 

# numInc, totalUpdate = outputResults(desFeat2) 
# totalClassified += totalUpdate 
# numIncorrectClass += numInc 
# print(numIncorrectClass) 
# print(totalClassified) 

# print(numIncorrect) 
genreArray = [] 
#     genreDict = {} 
# print(genreStrings[i]) 
makeGenreDict("SpotifyFeatures.csv", genreArray) 
clf = tree.DecisionTreeClassifier()
# print(numericalFeatures[0])
# print(genreArray)
clf = clf.fit(numericalFeatures, genreArray)
print("fitted the tree")
# dot_data = tree.export_graphviz(clf, out_file="graph") 
# graph = graphviz.Source(dot_data) 
# graph.render("iris") 
# tree.plot_tree(clf) 
# dot_data = tree.export_graphviz(clf, out_file=None) 


Reading File Data!

fitted the tree


In [2]:
from sklearn import tree
from sklearn.externals.six import StringIO
print("before dot")
dotfile = StringIO()
dot_data = tree.export_graphviz(clf, out_file=dotfile)

print("after dot")
# Convert to png using system command (requires Graphviz)


# from subprocess import call
# call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])
print("here")
# Display in jupyter notebook
# from IPython.display import Image
# Image(filename = 'tree.png')
# graph = graphviz.Source(dot_data)
# print(graph)
# graph.render("iris") 
# dot_data = StringIO() 
# print("after dot tree")
# tree.export_graphviz(clf, out_file=dot_data) 
# print("after graphviz")
# graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
# print("before png")
# a=graph.write_png("tree.png") 
# print("hrere")
# clf.predict()


before dot
after dot
here


In [ ]:
print("starting")
graph = pydot.graph_from_dot_data(dotfile.getvalue())
print("almost there")
graph[0].write_png('CART.png')
print("here")

starting
almost there


In [ ]:
numInc, totalUpdate = outputResults(clf, desFeat2)
totalClassified += totalUpdate 
numIncorrectClass += numInc 
print(float(numIncorrectClass)/float(totalClassified))
